#Criar um script que faça:  
#Pegue as nfe em xml do email  
#Extrai os dados necessarios   
#Tratar dados   
#Remover materiais pesado E outras notas 
-TABUAS CABOQUINHO  
-CIMENTRO VOTORAN  
-Magalu   

#Aplicador formula de taxas  
#Inserir no template dos marketplaces

Ultilizar api do bling para cadastro de itens   

In [ ]:
import os
import shutil
import xml.etree.ElementTree as ET
import pandas as pd
from imap_tools import MailBox, AND
from env import pwd, user
from datetime import date, datetime
import requests
from bs4 import BeautifulSoup
import json
import time


In [ ]:
#Baixa os anexos XML dos emails

save_folder = "notas/nfes/"
os.makedirs(save_folder, exist_ok=True)

with MailBox("imap.gmail.com").login(user, pwd, initial_folder="INBOX", ) as mailbox:
    list_mail = mailbox.fetch(criteria=AND(date=date.today())

    )
    for email in list_mail:
        for anexo in email.attachments:
            if  anexo.filename.lower().endswith(".xml"):
                file_path = os.path.join(save_folder, anexo.filename)
                if not os.path.exists(file_path):   
                    with open(file_path, "wb") as f:
                        f.write(anexo.payload)

In [ ]:
# Salva os XMLs em pastas por Emitente e extrai os produtos para um Excel

pasta_origem = "notas/nfes"
todos_produtos = []


def extrai_dados(caminho_arquivo):
    produtos = []
    tree = ET.parse(caminho_arquivo)
    root = tree.getroot()
    ns = {"ns": "http://www.portalfiscal.inf.br/nfe"}
    data_str = root.find(".//ns:ide/ns:dhEmi", ns).text
    data_emissao = datetime.fromisoformat(data_str)

    
    natOp = root.find(".//ns:ide/ns:natOp", ns)
    if natOp is not None and natOp.text.strip().upper() == 'BONIFICACAO, DOACAO OU BRINDE':
        return [] 
    
    for det in root.findall(".//ns:det", ns):   
        produto = {}
        temp_codigo = det.find("./ns:prod/ns:cProd", ns).text
        if not "-" in temp_codigo:
            produto ['Codigo Produto'] = temp_codigo
        produto ['Descrição'] = det.find("./ns:prod/ns:xProd", ns).text
        produto ['Valor_unitário'] = det.find("./ns:prod/ns:vUnCom", ns).text
        produto ['Código de Barras'] = det.find("./ns:prod/ns:cEAN", ns).text 
        produto ['Sku'] = produto['Código de Barras']
        if produto['Sku'] == 'SEM GTIN':
            produto['Sku'] = produto['Descrição']
        produto ['Fornecedor'] = root.find(".//ns:emit/ns:xNome", ns).text
        produto ['Data Emissão'] = data_emissao

        produtos.append(produto)
    return produtos


# Percorre todos os arquivos da pasta
for arquivo in os.listdir(pasta_origem):
    if arquivo.lower().endswith(".xml"):
        caminho_arquivo = os.path.join(pasta_origem, arquivo)

        try:
            # Abre e parseia o XML
            tree = ET.parse(caminho_arquivo)
            root = tree.getroot()

            # Define namespace
            ns = {"ns": "http://www.portalfiscal.inf.br/nfe"}

            # Busca o CNPJ do emitente
            cnpj_emitente = root.find(".//ns:emit/ns:CNPJ", ns)
            cnpj_emitente
            if cnpj_emitente is None:
                print(f"⚠️ CNPJ não encontrado em {arquivo}")
                continue

            cnpj_emitente = cnpj_emitente.text

            produtos = extrai_dados(caminho_arquivo)
            todos_produtos.extend(produtos)              


            emitente = root.find(".//ns:emit/ns:xNome", ns).text
            
            pasta_destino = os.path.join("notas/")

            # Cria a pasta se não existir
            os.makedirs(pasta_destino, exist_ok=True)

            # Copia o arquivo para a pasta
            shutil.copy(caminho_arquivo, pasta_destino)

        except Exception as e:
            print(f" {arquivo}: {e}")



fornecedores_pesados = [
    'IND E COM DE TUBOS E CONEXOES FORT.COM',
    'VOTORANTIM CIMENTOS SA',
    'CABOQUINHO MATERIAIS PARA CONSTRUCAO'
]


produtos = pd.DataFrame(todos_produtos)
produtos = produtos[~produtos['Fornecedor'].isin(fornecedores_pesados)]
produtos = produtos.sort_values(by="Data Emissão", ascending=False)

produtos = produtos.drop_duplicates(subset='Codigo Produto', keep='first')



In [ ]:
produtos['Fornecedor'].unique()



In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
}



In [ ]:
for  index, produto in produtos.iterrows():
    fornecedor = produto['Fornecedor']
    codigo_produto = produto['Codigo Produto']
    descricao = produto['Descrição']
    try:
        if fornecedor == 'CONSTRUDIGI DISTRIBUIDORA DE MATERIAIS PARA CONSTRUCAO LTDA':   
            url = f'https://www.construdigi.com.br/produto/{codigo_produto}/{descricao}'
        elif fornecedor == 'M.S.B. COMERCIO DE MATERIAIS PARA CONSTRUCAO':
            url = f'https://msbitaqua.com.br/produto/{codigo_produto}/{descricao}'
        elif fornecedor == "CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU":
            url = f'https://www.construja.com.br/produto/{codigo_produto}/{descricao}'
        else:
            print('Indisponivel')
            continue
        
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        script = soup.find("script", type="application/json")
        data = json.loads(script.string) if script else {}

        extrai = data.get("props", {}).get("pageProps", {}).get("produto", {})
        url_img = data.get("props", {}).get("pageProps", {}).get("seo", {}).get("imageUrl", 'Não disponivel')
        marca = next((p.get("desc") for p in extrai.get("dimensoes", []) if p.get("label") == "MARCA"), "Não disponível")
        peso = extrai.get("pesoBruto", "Não disponível")
        if codigo_barras == 'SEM GTIN':
            codigo_barras = extrai.get("codBarra", "SEM GTIN")
        produtos.at[index, 'Código de Barras'] = codigo_barras
        produtos.at[index, 'Peso'] = peso
        produtos.at[index, "Marca"] = marca
        produtos.at[index, 'Url Imagem'] = url_img
        
        print(produto['Descrição'])
        
    except Exception as e:
        print(e)


produtos.to_excel('produtos.xlsx', index=False)


In [ ]:
import json
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright




async def run():

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, wait_until="networkidle")

        # pega o HTML renderizado
        html = await page.content()
        soup = BeautifulSoup(html, "html.parser")
        # tenta pegar o JSON do __NEXT_DATA__
        script = soup.find("script", id="__NEXT_DATA__")
        if script:
            data = json.loads(script.string)
            produto = data.get("props", {}).get("pageProps", {}).get("produto", {})
            url_img = data.get("props", {}).get("pageProps", {}).get("seo", {}).get("imageUrl", 'Não disponivel')
            print(url_img)
            print("Marca:", next((p.get("desc") for p in produto.get("dimensoes", []) if p.get("label") == "MARCA"), "não disponível"))
            print("Peso:", produto.get("pesoBruto", "não disponível"))
            print("Código de Barras:", produto.get("codBarra", "não disponível"))

        else:
            print("❌ Nenhum <script id=__NEXT_DATA__> encontrado")

        await browser.close()




In [ ]:
import requests
from bs4 import BeautifulSoup
import json

response = requests.get('https://msbitaqua.com.br/produto/11748/11748')
soup = BeautifulSoup(response.text, "html.parser")
script = soup.find("script", type="application/json")
data = json.loads(script.string)

produto = data['props']['pageProps']['produto']
produto

In [ ]:
import json
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright

async def run():

    async with async_playwright() as p:
        url = 'https://msbitaqua.com.br/produto/2123/2123'
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, wait_until="networkidle")

        # pega o HTML renderizado
        html = await page.content()
        soup = BeautifulSoup(html, "html.parser")
        # tenta pegar o JSON do __NEXT_DATA__
        script = soup.find("script", id="__NEXT_DATA__")
        if script:
            data = json.loads(script.string)
            print(data.keys())
            produto = data.get("props", {}).get("pageProps", {}).get("produto", {})
            url_img = data.get("props", {}).get("pageProps", {}).get("seo", {}).get("imageUrl", 'Não disponivel')
            print(url_img)
            print("Marca:", next((p.get("desc") for p in produto.get("dimensoes", []) if p.get("label") == "MARCA"), "não disponível"))
            print("Peso:", produto.get("pesoBruto", "não disponível"))
            print("Código de Barras:", produto.get("codBarra", "não disponível"))
        else:
            print("❌ Nenhum <script id=__NEXT_DATA__> encontrado")

        await browser.close()

# 👉 Em Jupyter/IPython:


await run()

In [ ]:
import os
import shutil
import xml.etree.ElementTree as ET
import pandas as pd
from imap_tools import MailBox, AND
from env import pwd, user
from datetime import date, datetime
import requests
from bs4 import BeautifulSoup
import json
import time


import json
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright

async def run():

    async with async_playwright() as p:
        url = 'https://www.construdigi.com.br/produto/10422/multilit-tubo-pvc-esg-75'
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, wait_until="networkidle")

        # pega o HTML renderizado
        html = await page.content()
        soup = BeautifulSoup(html, "html.parser")
        # tenta pegar o JSON do __NEXT_DATA__
        script = soup.find("script", id="__NEXT_DATA__")
        if script:
            data = json.loads(script.string)
            produto = data.get("props", {}).get("pageProps", {}).get("produto", {})
            url_img = data.get("props", {}).get("pageProps", {}).get("seo", {}).get("imageUrl", 'Não disponivel')
            print(url_img)
            print("Marca:", next((p.get("desc") for p in produto.get("dimensoes", []) if p.get("label") == "MARCA"), "não disponível"))
            print("Peso:", produto.get("pesoBruto", "não disponível"))
            print("Código de Barras:", produto.get("codBarra", "não disponível"))
        else:
            print("❌ Nenhum <script id=__NEXT_DATA__> encontrado")

        await browser.close()

# 👉 Em Jupyter/IPython:


await run()


In [ ]:
import pandas as pd
from rapidfuzz import fuzz, process

produtos = pd.read_excel('produtos.xlsx')

corantes = produtos.query('Descrição == ')

# # Comparar descrições e achar semelhantes
# produtos['Grupo'] = None
# for i, desc in enumerate(produtos['Descrição']):
#     match = process.extractOne(desc, produtos['Descrição'], scorer=fuzz.token_sort_ratio)
#     if match[1] > 85 and match[0] != desc:
#         produtos.at[i, 'Grupo'] = match[0]



produtos



In [ ]:
import pandas as pd
df = pd.read_excel('produtos.xlsx')

from rapidfuzz import process, fuzz

# Criar uma cópia do dataframe para trabalhar
df_fuzzy = df.copy()

# Criar um dicionário para armazenar grupos de produtos semelhantes
grupos = {}
grupo_id = 1

# Threshold de similaridade (ajustável)
threshold = 75

# Lista para armazenar o grupo de cada item
grupo_coluna = []

for i, desc in enumerate(df_fuzzy['Descrição']):
    # Verificar se já foi agrupado
    if any(desc in v for v in grupos.values()):
        for gid, itens in grupos.items():
            if desc in itens:
                grupo_coluna.append(gid)
                break
        continue
    
    # Criar novo grupo
    grupos[grupo_id] = [desc]
    grupo_coluna.append(grupo_id)
    
    # Procurar similares
    matches = process.extract(desc, df_fuzzy['Descrição'], scorer=fuzz.token_sort_ratio, limit=None)
    for match_desc, score, idx in matches:
        if score >= threshold and match_desc != desc:
            grupos[grupo_id].append(match_desc)
    
    grupo_id += 1

# Adicionar coluna com id do grupo
df_fuzzy['Grupo'] = grupo_coluna

print(df_fuzzy.head(50))
df_fuzzy.to_excel('Fuzzy.xlsx', index=False)





Mercado LIVRE:50% do valor do produto (para produtos até R$ 12,50), R$ 6,25 (entre R$ 12,50 e R$ 29), R$ 6,50 (entre R$ 29 e R$ 50) e R$ 6,75 (entre R$ 50 e R$ 79)

Comissão de Categoria Construção 12%


SHOPE taxa de transação é sobre o total do produto então é 2% de taxa e 18 de comissão


In [4]:
import os
import xml.etree.ElementTree as ET

def extrair_fornecedores(pasta_origem):
    ns = {'nfe': 'http://www.portalfiscal.inf.br/nfe'}
    fornecedores = []

    for arquivo in os.listdir(pasta_origem):
        if arquivo.lower().endswith(".xml"):
            caminho_arquivo = os.path.join(pasta_origem, arquivo)
            try:
                tree = ET.parse(caminho_arquivo)
                root = tree.getroot()

                emitente = root.find('.//nfe:emit', ns)
                if emitente is not None:
                    fornecedor = {
                        'CNPJ': emitente.findtext('nfe:CNPJ', default='', namespaces=ns),
                        'RazaoSocial': emitente.findtext('nfe:xNome', default='', namespaces=ns),
                        'NomeFantasia': emitente.findtext('nfe:xFant', default='', namespaces=ns),
                        'Logradouro': emitente.findtext('nfe:enderEmit/nfe:xLgr', default='', namespaces=ns),
                        'Numero': emitente.findtext('nfe:enderEmit/nfe:nro', default='', namespaces=ns),
                        'Bairro': emitente.findtext('nfe:enderEmit/nfe:xBairro', default='', namespaces=ns),
                        'Municipio': emitente.findtext('nfe:enderEmit/nfe:xMun', default='', namespaces=ns),
                        'UF': emitente.findtext('nfe:enderEmit/nfe:UF', default='', namespaces=ns),
                        'CEP': emitente.findtext('nfe:enderEmit/nfe:CEP', default='', namespaces=ns),
                        'Telefone': emitente.findtext('nfe:enderEmit/nfe:fone', default='', namespaces=ns),
                        'IE': emitente.findtext('nfe:IE', default='', namespaces=ns),
                    }

                    # Verifica se o fornecedor já existe na lista (pelo CNPJ)
                    if not any(f['CNPJ'] == fornecedor['CNPJ'] for f in fornecedores):
                        fornecedores.append(fornecedor)

            except ET.ParseError:
                print(f"Erro ao ler o XML: {arquivo}")
            except Exception as e:
                print(f"Erro inesperado em {arquivo}: {e}")

    return fornecedores


# Exemplo de uso
if __name__ == "__main__":
    pasta_origem = "notas/nfes/"  # coloque o caminho da pasta
    lista_fornecedores = extrair_fornecedores(pasta_origem)

    print(f"Foram encontrados {len(lista_fornecedores)} fornecedores únicos:\n")
    for f in lista_fornecedores:
        print(f)


lista_fornecedores
df = pd.DataFrame(lista_fornecedores)
df.to_csv('fornecedores.csv', index=False)

Foram encontrados 17 fornecedores únicos:

{'CNPJ': '13571969000164', 'RazaoSocial': 'MJR CUNHA DISTRIBUIDORA DE MAT.CONSTR LTDA', 'NomeFantasia': 'MEGALESTE', 'Logradouro': 'ESTRADA MUNICIPAL', 'Numero': '155', 'Bairro': 'JD SANTO AFONSO', 'Municipio': 'GUARULHOS', 'UF': 'SP', 'CEP': '07215040', 'Telefone': '1120854822', 'IE': '336578097112'}
{'CNPJ': '38203316000234', 'RazaoSocial': 'CONSTRUDIGI DISTRIBUIDORA DE MATERIAIS PARA CONSTRUCAO LTDA', 'NomeFantasia': 'CONSTRUDIGI SP', 'Logradouro': 'ESTRADA VOVO CAROLINA', 'Numero': '1715', 'Bairro': 'GUAIANAZES', 'Municipio': 'SAO PAULO', 'UF': 'SP', 'CEP': '08473370', 'Telefone': '1127322007', 'IE': '130957633115'}
{'CNPJ': '03928279000166', 'RazaoSocial': 'M.S.B. COMERCIO DE MATERIAIS PARA CONSTRUCAO', 'NomeFantasia': 'MSB DISTRIBUIDORA', 'Logradouro': 'AVENIDA ROCHEDO DE MINAS', 'Numero': '49', 'Bairro': 'PARQUE VIVIANE', 'Municipio': 'ITAQUAQUECETUBA', 'UF': 'SP', 'CEP': '08582005', 'Telefone': '1146490720', 'IE': '379147716114'}
{'CNP

In [ ]:
import pandas as pd

# Caminho do novo arquivo CSV
arquivo_csv = "fornecedores.csv"

# Ler o arquivo CSV e mostrar as colunas
df = pd.read_csv(arquivo_csv, sep=None, engine="python")  # detecta separador automaticamente
df.columns.tolist()




['\ufeff"ID"',
 'Código',
 'Nome',
 'Fantasia',
 'Endereço',
 'Número',
 'Complemento',
 'Bairro',
 'CEP',
 'Cidade',
 'UF',
 'Contatos',
 'Fone',
 'Fax',
 'Celular',
 'E-mail',
 'Web Site',
 'Tipo pessoa',
 'CNPJ / CPF',
 'IE / RG',
 'IE isento',
 'Situação',
 'Observações',
 'Estado civil',
 'Profissão',
 'Sexo',
 'Data nascimento',
 'Naturalidade',
 'Nome pai',
 'CPF pai',
 'Nome mãe',
 'CPF mãe',
 'Segmento',
 'Vendedor',
 'Tipo contato',
 'E-mail para envio NFe',
 'Limite de crédito',
 'Cliente desde',
 'Próxima visita',
 'Condição de pagamento',
 'Regime tributário']

In [12]:
import csv
import os
import re

# Lista completa de contatos
dados = [
    {'CNPJ': '13571969000164', 'RazaoSocial': 'MJR CUNHA DISTRIBUIDORA DE MAT.CONSTR LTDA', 'NomeFantasia': 'MEGALESTE', 'Logradouro': 'ESTRADA MUNICIPAL', 'Numero': '155', 'Bairro': 'JD SANTO AFONSO', 'Municipio': 'GUARULHOS', 'UF': 'SP', 'CEP': '07215040', 'Telefone': '1120854822', 'IE': '336578097112'},
    {'CNPJ': '38203316000234', 'RazaoSocial': 'CONSTRUDIGI DISTRIBUIDORA DE MATERIAIS PARA CONSTRUCAO LTDA', 'NomeFantasia': 'CONSTRUDIGI SP', 'Logradouro': 'ESTRADA VOVO CAROLINA', 'Numero': '1715', 'Bairro': 'GUAIANAZES', 'Municipio': 'SAO PAULO', 'UF': 'SP', 'CEP': '08473370', 'Telefone': '1127322007', 'IE': '130957633115'},
    {'CNPJ': '03928279000166', 'RazaoSocial': 'M.S.B. COMERCIO DE MATERIAIS PARA CONSTRUCAO', 'NomeFantasia': 'MSB DISTRIBUIDORA', 'Logradouro': 'AVENIDA ROCHEDO DE MINAS', 'Numero': '49', 'Bairro': 'PARQUE VIVIANE', 'Municipio': 'ITAQUAQUECETUBA', 'UF': 'SP', 'CEP': '08582005', 'Telefone': '1146490720', 'IE': '379147716114'},
    {'CNPJ': '04648461000207', 'RazaoSocial': 'CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU', 'NomeFantasia': 'ATACADAO PALANQUE', 'Logradouro': 'ESTRADA VOVO CAROLINA', 'Numero': '777', 'Bairro': 'JD. PALANQUE', 'Municipio': 'SAO PAULO', 'UF': 'SP', 'CEP': '08473370', 'Telefone': '1127322007', 'IE': '116629720112'},
    {'CNPJ': '41861211000104', 'RazaoSocial': 'SOOPER HOLDING LTDA', 'NomeFantasia': '', 'Logradouro': 'Av. Ariston Azevedo', 'Numero': '75', 'Bairro': 'Catumbi', 'Municipio': 'Sao Paulo', 'UF': 'SP', 'CEP': '03021020', 'Telefone': '', 'IE': '131584450111'},
    {'CNPJ': '04064707000112', 'RazaoSocial': 'DENVER IMPERMEABILIZANTES, INDUSTRIA E C', 'NomeFantasia': 'IMPER', 'Logradouro': 'R. VER. JOAO B. FITIPALDI', 'Numero': '500', 'Bairro': 'VILA MALUF', 'Municipio': 'SUZANO', 'UF': 'SP', 'CEP': '08685000', 'Telefone': '1147416000', 'IE': '672148028119'},
    {'CNPJ': '61064838016137', 'RazaoSocial': 'Saint-Gobain do Brasil Produtos Ind ustriais e para Construc', 'NomeFantasia': 'SG do Brasil Jacarei - SP', 'Logradouro': 'Avenida Getúlio Dorneles Vargas', 'Numero': '3099', 'Bairro': 'Jardim California', 'Municipio': 'JACAREI', 'UF': 'SP', 'CEP': '12305010', 'Telefone': '', 'IE': '392291700114'},
    {'CNPJ': '01637895002267', 'RazaoSocial': 'VOTORANTIM CIMENTOS SA', 'NomeFantasia': 'VC - CD S.MIGUEL PAULISTA', 'Logradouro': 'AV DOUTOR JOSE ARTUR DA NOVA', 'Numero': '951', 'Bairro': 'SAO MIGUEL PAULISTA', 'Municipio': 'SAO PAULO', 'UF': 'SP', 'CEP': '08090000', 'Telefone': '1169562000', 'IE': '148723661112'},
    {'CNPJ': '01637895026280', 'RazaoSocial': 'VOTORANTIM CIMENTOS SA', 'NomeFantasia': 'VC - CD GUARULHOS', 'Logradouro': 'R DOUTOR WAGNER ANTONIO ALBUQUERQUE', 'Numero': 'S/N', 'Bairro': 'CIDADE INDUSTRIAL SATELITE DE SAO P', 'Municipio': 'GUARULHOS', 'UF': 'SP', 'CEP': '07230630', 'Telefone': '4133885151', 'IE': '127594490110'},
    {'CNPJ': '20812457000108', 'RazaoSocial': 'IND E COM DE TUBOS E CONEXOES FORT.COM', 'NomeFantasia': 'TUBOS E CONEXOES FORT.COM', 'Logradouro': 'Rua Jacinto José Fagundes', 'Numero': '14', 'Bairro': 'Vila Pedroso', 'Municipio': 'São Paulo', 'UF': 'SP', 'CEP': '08070530', 'Telefone': '', 'IE': '143804974119'},
    {'CNPJ': '61064838003400', 'RazaoSocial': 'Saint-Gobain do Brasil Produtos Ind ustriais e para Construc', 'NomeFantasia': '', 'Logradouro': 'Via de acesso Joao de Goes', 'Numero': '2127', 'Bairro': 'JD ALVORADA', 'Municipio': 'JANDIRA', 'UF': 'SP', 'CEP': '06612000', 'Telefone': '1121968000', 'IE': '398096510118'},
    {'CNPJ': '61064838017028', 'RazaoSocial': 'Saint-Gobain do Brasil Produtos Ind ustriais e para Construc', 'NomeFantasia': 'SG do Brasil  São Paulo - SP', 'Logradouro': 'Av. Condessa Elisabeth de Robiano', 'Numero': '320', 'Bairro': 'Catumbi', 'Municipio': 'SAO PAULO', 'UF': 'SP', 'CEP': '03021055', 'Telefone': '', 'IE': '133937536113'},
    {'CNPJ': '06293564000146', 'RazaoSocial': 'KADESH EQUIPAMENTOS PROFISSIONAIS LTDA', 'NomeFantasia': 'KADESH EQUIPAMENTOS PROFISSIONAIS LTDA', 'Logradouro': 'RUA ANTONIO LOURENCO', 'Numero': '08', 'Bairro': 'CENTRO', 'Municipio': 'IMBITUVA', 'UF': 'PR', 'CEP': '84430000', 'Telefone': '4234368950', 'IE': '9034773693'},
    {'CNPJ': '03010739000172', 'RazaoSocial': 'Global Center Com.Ferragens Ltda', 'NomeFantasia': 'Global Atacadista', 'Logradouro': 'R CHIPUE', 'Numero': '117', 'Bairro': 'VL D EIRAS', 'Municipio': 'São Paulo', 'UF': 'SP', 'CEP': '08010260', 'Telefone': '1120303000', 'IE': '115420623115'},
    {'CNPJ': '17467250000139', 'RazaoSocial': 'CABOQUINHO MATERIAIS PARA CONSTRUCAO', 'NomeFantasia': 'CABOQUINHO', 'Logradouro': 'AVENIDA ROCHEDO DE MINAS', 'Numero': '47', 'Bairro': 'PARQUE VIVIANE', 'Municipio': 'ITAQUAQUECETUBA', 'UF': 'SP', 'CEP': '08582005', 'Telefone': '1146490720', 'IE': '379116068118'},
]

colunas_bling = [
    "ID","Código","Nome","Fantasia","Endereço","Número","Complemento","Bairro","CEP","Cidade",
    "UF","Contatos","Fone","Fax","Celular","E-mail","Web Site","Tipo pessoa","CNPJ / CPF",
    "IE / RG","IE isento","Situação","Observações","Estado civil","Profissão","Sexo","Data nascimento",
    "Naturalidade","Nome pai","CPF pai","Nome mãe","CPF mãe","Segmento","Vendedor","Tipo contato",
    "E-mail para envio NFe","Limite de crédito","Cliente desde","Próxima visita","Condição de pagamento","Regime tributário"
]

arquivo_csv = "contatos_bling_completo.csv"

def normalizar_cnpj(cnpj):
    return re.sub(r'\D', '', cnpj)

def formatar_cnpj(cnpj):
    cnpj_n = normalizar_cnpj(cnpj)
    return f"{cnpj_n[:2]}.{cnpj_n[2:5]}.{cnpj_n[5:8]}/{cnpj_n[8:12]}-{cnpj_n[12:]}"

def criar_linha(item):
    cnpj_formatado = formatar_cnpj(item.get("CNPJ", ""))
    return {
        "ID": "",
        "Código": "",
        "Nome": item.get("RazaoSocial", ""),
        "Fantasia": item.get("NomeFantasia", ""),
        "Endereço": item.get("Logradouro", ""),
        "Número": item.get("Numero", ""),
        "Complemento": "",
        "Bairro": item.get("Bairro", ""),
        "CEP": item.get("CEP", ""),
        "Cidade": item.get("Municipio", ""),
        "UF": item.get("UF", ""),
        "Contatos": "",
        "Fone": item.get("Telefone", ""),
        "Fax": "",
        "Celular": "",
        "E-mail": "",
        "Web Site": "",
        "Tipo pessoa": "Pessoa Jurídica",
        "CNPJ / CPF": cnpj_formatado,
        "IE / RG": item.get("IE", ""),
        "IE isento": "N",
        "Situação": "Ativo",
        "Observações": "",
        "Estado civil": "",
        "Profissão": "",
        "Sexo": "",
        "Data nascimento": "",
        "Naturalidade": "",
        "Nome pai": "",
        "CPF pai": "",
        "Nome mãe": "",
        "CPF mãe": "",
        "Segmento": "",
        "Vendedor": item.get("NomeFantasia", "") or item.get("RazaoSocial"),
        "Tipo contato": "Fornecedor",
        "E-mail para envio NFe": "",
        "Limite de crédito": "",
        "Cliente desde": "",
        "Próxima visita": "",
        "Condição de pagamento": "",
        "Regime tributário": ""
    }

# Carrega CSV existente
csv_existente = {}
if os.path.exists(arquivo_csv):
    with open(arquivo_csv, 'r', encoding='utf-8-sig', newline='') as f:
        reader = csv.DictReader(f)
        for row in reader:
            cnpj_norm = normalizar_cnpj(row["CNPJ / CPF"])
            csv_existente[cnpj_norm] = row

# Adiciona ou atualiza
atualizados = 0
novos = 0

for item in dados:
    cnpj_norm = normalizar_cnpj(item["CNPJ"])
    nova_linha = criar_linha(item)
    
    if cnpj_norm in csv_existente:
        linha_atual = csv_existente[cnpj_norm]
        mudou = False
        campos_verificar = ["Nome", "Fantasia", "Endereço", "Número", "Bairro", "CEP", "Cidade", "UF", "Fone", "IE / RG", "Vendedor", "Situação", "Tipo pessoa", "Tipo contato"]
        for campo in campos_verificar:
            if linha_atual.get(campo, "") != nova_linha[campo]:
                linha_atual[campo] = nova_linha[campo]
                mudou = True
        if mudou:
            atualizados += 1
    else:
        csv_existente[cnpj_norm] = nova_linha
        novos += 1

# Salva CSV final
with open(arquivo_csv, 'w', encoding='utf-8-sig', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=colunas_bling)
    writer.writeheader()
    for linha in csv_existente.values():
        writer.writerow(linha)

print(f"✅ {novos} novos contatos adicionados e {atualizados} contatos atualizados no '{arquivo_csv}'!")



✅ 15 novos contatos adicionados e 0 contatos atualizados no 'contatos_bling_completo.csv'!
